### Импорт библиотек

In [3]:
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request, sys, time
import requests
from razdel import sentenize
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import contractions
import re
import tqdm
import unicodedata
from random import seed
import spacy
import ru_core_news_md


import syllables
import rusyllab
import seaborn as sns
%matplotlib inline

from mlxtend.regressor import StackingRegressor
from mlxtend.data import boston_housing_data
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.neighbors import KNeighborsRegressor

from nltk import sent_tokenize, word_tokenize
import holidays

import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('russian'))
nltk.download('wordnet')
nltk.download('stopwords');
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from tqdm.contrib import tzip
from time import sleep
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from pymystem3 import Mystem
from datetime import datetime

from tqdm import tqdm
tqdm.pandas() 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats
from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from xgboost import plot_importance
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVR

from mlxtend.regressor import StackingCVRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
from math import sqrt
from sklearn.metrics import r2_score
import lightgbm as ltb
import ast

ru_holidays = holidays.country_holidays('RU')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\steam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\steam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\steam\anaconda3\lib\site-packages\tqdm\std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### Генерация признаков по дате

In [2]:
def week_day(datestring):
    dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
    return dt.weekday()

def hour(datestring):
    dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
    return dt.hour

def is_weekend(datestring):
    dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
    if dt.weekday() in [5,6]:
        return 1
    else:
        return 0

def is_holiday(datestring):
    dt = datetime.strptime(datestring, '%Y-%m-%d %H:%M:%S')
    if dt.date() in ru_holidays:
        return 1
    else:
        return 0
    
def get_id(idd, ses):
    return idd[:idd.find(ses)]

### Подключение к статье и получение информации

In [3]:
def get_text_info(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    page = requests.get(url, headers=headers)
    
    
    soup = BeautifulSoup(page.text, "html.parser")
    paragraphs, words = 0, 0
    token_paragraphs, count_sents = [], []
    full_text = ''
    for s in soup.find_all('p'):
        if 'div' not in str(s) and any(c.isalpha() for c in s.text):
            paragraphs += 1
            tokens = word_tokenize(re.sub(r'[^а-яА-Яa-zA-z\s]', '', s.text, re.I|re.A))
            words += len(tokens)
            token_paragraphs.append(tokens)
            sentences = list(sentenize(s.text))
            count_sents.append((len(sentences)))
            
            for sentence in sentences:
                full_text += sentence.text + ' '
            

    #Средняя длина предложения
    avg_par_len = np.mean([len(tokens) for tokens in token_paragraphs]) if paragraphs !=0 else 0.0

    #Количество предложений
    sentence_count =  sum(count_sents) if paragraphs !=0 else 0
    
    #Доп категории
    s = soup.find('div', {"class": "article__header__info-block"})
    category = s.find('a', {"class": "article__header__category"}).contents[0]
    
    s = soup.find('div', {"class": "article__tags__container"})
    
    #Тэги
    if s:
        tags = [tag for tag in s.text.split('\n') if tag]
    else:
        tags = []
    
    #-------------------------------------КОНТЕНТ----------------------------------------------
    
    soup = BeautifulSoup(page.content, "html.parser")
    
    #Заголовочное изображение
    if soup.find_all('div', {"class": "article__main-image__wrap"}):
        title_image = 1
    else:
        title_image = 0
        
    #Количество видео
    video_count = len(soup.find_all('div', {"class": "article__inline-video js-insert-video-container"}))
    
    #Количество изображений, помимо заголовочного изображения
    gallery_count = len(soup.find_all('div', {"class": ["gallery_vertical__image-wrap",  "gallery js-gallery"]}))
    pictures = soup.find_all('div', {"class": ["article__picture"]})
    
    if pictures:
        pics_count, pics_margin  = 0, 0
        for pic in pictures:
            if str(pic).startswith('<div class="article__picture article__picture_margin">'):
                pics_margin += 1
            else:
                pics_count += 1
        pics_count += int(pics_margin/2) 
    else:
        pics_count = 0

    image_count = pics_count + gallery_count
    
    
    return category, tags, full_text, paragraphs, words, avg_par_len, sentence_count, title_image, video_count, image_count

### Предобработка текста

In [4]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS

functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NPRO', 'ADVB'}  # function words

def del_tags_from_text(title):
    if title.find('\n') != -1:
        return title[:title.find('\n')]
    else:
        return title

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def pre_process_corpus(docs):
    norm_docs = []
    for doc in docs:
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        #doc = remove_accented_chars(doc)
        doc = contractions.fix(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^а-яА-Я\s]', '', doc, re.I|re.A) #0-9
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)

    return norm_docs


def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS



m = Mystem()
morf = pymorphy2.MorphAnalyzer()

def get_lem(sentence):

    #lemmas = m.lemmatize(sentence)
    #lemmas = [lemma for lemma in lemmas if (lemma not in stopwords and len(lemma) > 1)]
    #lemmas = [word for word in lemmas if pos(word) not in functors_pos]
    
    #FIX
    
    #tokens = nlp(sentence)
    #tokens = [token.lemma_ for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = word_tokenize(sentence)
    
    tokens = [word for word in tokens if pos(word) not in functors_pos]
    
    tokens = [morf.parse(token)[0].normal_form for token in tokens]
    
    #text = " ".join(tokens)
    return tokens

### Подсчет важных слов названии статьи

In [5]:
def get_important_words(info_df, column, num_words = 100, count=19):
    views_info = pd.DataFrame(info_df.groupby('words')[column].mean())
    views_info = views_info.sort_values(by=[column], ascending=False).reset_index()
    views_info['count'] = views_info.apply(lambda row: word_voc[row.words], axis = 1)
    df_ = views_info.loc[views_info['count'] > count]
    return df_, list(df_.words)[:num_words]

In [6]:
def find_words_full(sentence):
    sentence = del_tags_from_text(sentence)
    preprocessed = pre_process_corpus([sentence])
    tokens = get_lem(preprocessed[0])
    #print(tokens)
    count = 0
    for token in tokens:
        if token in words_full:
            count += 1
    return count

def find_words_view(sentence):
    sentence = del_tags_from_text(sentence)
    preprocessed = pre_process_corpus([sentence])
    tokens = get_lem(preprocessed[0])
    #print(tokens)
    count = 0
    for token in tokens:
        if token in words_view:
            count += 1
    return count

### Применить функции к датафрейму

In [7]:
def get_final_df(df_train):
    #df_train['authors'] = ordinal_encoder_ath.transform(np.array(df_train['authors']).reshape(-1, 1)).astype(int)
    #df_train['category'] = ordinal_encoder_cat.transform(np.array(df_train['category']).reshape(-1, 1)).astype(int)
    df_train['day'] = pd.to_datetime(df_train['publish_date']).dt.strftime("%d").astype(int)
    df_train['month'] = pd.to_datetime(df_train['publish_date']).dt.strftime("%m").astype(int)
    #df_train['weekday'] = df_train['publish_date'].apply(week_day)
    df_train['hour'] = df_train['publish_date'].apply(hour)
    df_train['weekend'] = df_train['publish_date'].apply(is_weekend)
    df_train['holiday'] = df_train['publish_date'].apply(is_holiday)
    #df_view, words_view = get_important_words(info, column='views', num_words = 100, count=10)
    #df_full, words_full = get_important_words(info, column='full_reads_percent', num_words = 50, count=20)
    df_train['full_affect'] = df_train['title'].apply(find_words_full)
    df_train['view_affect'] = df_train['title'].apply(find_words_view)
    #df_train = df_train.drop(columns=['publish_date', 'title'])
    #print('ok')
    df_train['id'] = df_train.apply(lambda df_train: get_id(df_train['document_id'], df_train['session']), axis=1)
    df_train['URL'] = df_train['id'].apply(lambda x: 'http://rbc.ru/rbcfreenews/'  + x)
    df_train = df_train.drop(columns=['id'])
    df_train['cat'], df_train['tags'], df_train['text'], df_train['paragraphs'], df_train['words'], df_train['avg_par_len'], df_train['sentence_count'], df_train['title_image'], df_train['video_count'], df_train['image_count'] = zip(*df_train['URL'].progress_map(get_text_info))
    
    return df_train

### Слоги и буквы

In [8]:
def syllabs_letters(text):

    rus_text = re.sub(r'[^а-яА-Я\s]', '', text, re.I|re.A) #a-zA-z
    eng_text = re.sub(r'[^a-zA-z\s]', '', text, re.I|re.A)

    if bool(eng_text.replace(" ", "")):

        eng_syb_count = 0
        eng_syb_3_count = 0
        
        eng_tokens = eng_text.split()

        for word in eng_tokens:
            sbs = syllables.estimate(word)
            eng_syb_count += sbs
            
            if sbs > 2:
                eng_syb_3_count += 1
        
        eng_letters_count = len(eng_text.replace(" ", ""))
    else:
        eng_syb_count = 0
        eng_syb_3_count = 0
        eng_letters_count = 0

    if bool(rus_text.replace(" ", "")):
        
        rus_syb_count = 0
        rus_syb_3_count = 0
        
        rus_tokens = rus_text.split()
        rus_tokens = [token for token in rus_tokens if token not in stopwords]
        
        for word in rus_tokens:
            sbs = len(rusyllab.split_word(word))
            rus_syb_count += sbs
            
            if sbs > 2:
                rus_syb_3_count += 1           
                
        #syllab = rusyllab.split_words(rus_tokens)
        #syllab = [syl for syl in syllab if bool(syl.replace(" ", ""))]

        #rus_syb_count = len(syllab)
        rus_letters_count = len(rus_text.replace(" ", ""))
    else:
        rus_syb_count = 0
        rus_syb_3_count = 0
        rus_letters_count = 0

    syllabs = eng_syb_count + rus_syb_count
    letters = eng_letters_count + rus_letters_count
    syllabs_3 = eng_syb_3_count + rus_syb_3_count
    
    return syllabs, syllabs_3, letters

### Индексы читаемости

In [2]:
#где words – количество слов в тексте; 
#syllables – количество слогов в тексте; 
#sent – количество предложений в тексте
def Flesh_Kincaid_Grade(syllabes, words, sent):

    if words == 0 or sent == 0: return 0

    n = 0.318 * (float(words) / sent) + 14.2 * (float(syllabes) / words) - 30.5

    return n


#letters –число букв, 
#words – число слов,
#sent – число предложений
def Coleman_Liau(letters, words, sent):

    if words == 0: return 0

    n = 0.055 * (letters * (100/ words)) - 0.35 * (sent * (100 / words)) - 20.33

    return n

#где psyl – число слов с 3-мя и более слогами; 
#sent – число предложений
def SMOG(psyl, sent):
    
    if sent == 0: return 0
    
    n = 1.1 * np.sqrt((float(64.6) / sent) * psyl) + 0.05

    return n


#где n_letters – количество букв в тексте,
#n_words – количество слов,
#n_sent – количество предложений
def calc_ARI_index(n_letters, n_words, n_sent):

    if n_words == 0 or n_sent == 0: return 0

    n = 6.26 * (float(n_letters) / n_words) + 0.2805 * (float(n_words) / n_sent) - 31.04

    return n

In [10]:
def text_difficulty(df_train):

    df_train['FKG'] = df_train.progress_apply(lambda df_train: Flesh_Kincaid_Grade(df_train['syllabs'],
                                                                                   df_train['words'],
                                                                                   df_train['sentence_count']), axis=1)
    

    df_train['Coleman'] = df_train.progress_apply(lambda df_train: Coleman_Liau(df_train['letters'],
                                                                                   df_train['words'],
                                                                                   df_train['sentence_count']), axis=1)

    df_train['SMOG'] = df_train.progress_apply(lambda df_train: SMOG(df_train['syllabs3'],
                                                                        df_train['sentence_count']), axis=1)

    
    df_train['ARI'] = df_train.progress_apply(lambda df_train: calc_ARI_index(df_train['letters'],
                                                                                   df_train['words'],
                                                                                   df_train['sentence_count']), axis=1)
    
    return df_train
                            

In [11]:
df_train = pd.read_csv('train_dataset_train.csv')
df_test = pd.read_csv('test_dataset_test.csv')

### Получение важных слов в заголовке 

In [12]:
new_titles = df_train['title'].apply(del_tags_from_text)
new_titles = pre_process_corpus(new_titles)
new_titles = [get_lem(title) for title in tqdm(new_titles)]

100%|█████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:23<00:00, 296.17it/s]


In [14]:
words, views, full_reads_percent, depth = [], [], [], []

for i in tqdm(range(len(new_titles))):
    for word in new_titles[i]:
        words.append(word)
        views.append(df_train.views.iloc[i])
        full_reads_percent.append(df_train.full_reads_percent.iloc[i])
        depth.append(df_train.depth.iloc[i])

100%|████████████████████████████████████████████████████████████████████████████| 7000/7000 [00:01<00:00, 5481.96it/s]


In [15]:
word_voc = {}
for token in set(words):
    count = words.count(token)
    word_voc[token] = count
    
voc = {'words':words, 'views':views, 'full_reads_percent': full_reads_percent, 'depth':depth}
info = pd.DataFrame(voc)

In [16]:
info['count'] = info.apply(lambda row: word_voc[row.words], axis = 1)

In [17]:
info.head()

,words,views,full_reads_percent,depth,count
0,европейский,20460,35.85,1.134,15
1,банк,20460,35.85,1.134,80
2,развитие,20460,35.85,1.134,14
3,приостановить,20460,35.85,1.134,49
4,доступ,20460,35.85,1.134,21


In [18]:
df_view, words_view = get_important_words(info, column='views', num_words = 100, count=10)
df_full, words_full = get_important_words(info, column='full_reads_percent', num_words = 50, count=20)

In [ ]:
df_train_final = get_final_df(df_train)

In [ ]:
df_test_final = get_final_df(df_test)

### Применение tf-idf

In [ ]:
titles = df_train_final['text']
titles = pre_process_corpus(titles)
titles = [get_lem(title) for title in tqdm(titles)]

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=200)
X = vectorizer.fit_transform(titles)

In [ ]:
tf_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [ ]:
df_train = pd.concat([df_train_final, tf_df], axis= 1)

In [ ]:
X = vectorizer.transform(titles)
tf_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [ ]:
df_test = pd.concat([df_test_final, tf_df], axis= 1)

### Применить индексы читаемости и сохранить файлы

In [ ]:
df_train = text_difficulty(df_train)
df_test = text_difficulty(df_test)

In [ ]:
df_train = df_train.drop(columns=['title', 'text', 'URL'])
df_test = df_test.drop(columns=['title', 'text', 'URL'])

In [ ]:
df_train.to_csv('full_last_train.csv')
df_test.to_csv('full_last_test.csv')